<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML311Coursera747-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# Machine Learning Foundation

## Course 5, Part g: RNN DEMO


## Using RNNs to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).


In [1]:
%%capture
!pip install --upgrade tensorflow

In [2]:
from tensorflow import keras
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.datasets import imdb
from keras import initializers

In [3]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [4]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

25000 train sequences
25000 test sequences


In [5]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [6]:
x_train[123,:]  #Here's what an example sequence looks like

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99], dtype=int32)

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [7]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [8]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          1000000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 5)                 280       
                                                                 
 dense (Dense)               (None, 1)                 6         
                                                                 
Total params: 1000286 (3.82 MB)
Trainable params: 1000286 (3.82 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
rmsprop = keras.optimizers.legacy.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [11]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 3s 4ms/step - loss: 0.6553 - accuracy: 0.6188 - val_loss: 0.6019 - val_accuracy: 0.6861
Epoch 2/10
782/782 [==============================] - 3s 4ms/step - loss: 0.5521 - accuracy: 0.7270 - val_loss: 0.5350 - val_accuracy: 0.7317
Epoch 3/10
782/782 [==============================] - 3s 4ms/step - loss: 0.4889 - accuracy: 0.7710 - val_loss: 0.5003 - val_accuracy: 0.7514
Epoch 4/10
782/782 [==============================] - 3s 4ms/step - loss: 0.4481 - accuracy: 0.7924 - val_loss: 0.4814 - val_accuracy: 0.7624
Epoch 5/10
782/782 [==============================] - 3s 4ms/step - loss: 0.4197 - accuracy: 0.8079 - val_loss: 0.4632 - val_accuracy: 0.7749
Epoch 6/10
782/782 [==============================] - 3s 4ms/step - loss: 0.3989 - accuracy: 0.8215 - val_loss: 0.4538 - val_accuracy: 0.7810
Epoch 7/10
782/782 [==============================] - 3s 4ms/step - loss: 0.3833 - accuracy: 0.8296 - val_loss: 0.4484 - val_accuracy: 0.7858
Epoch 

In [12]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 1s 781us/step - loss: 0.4481 - accuracy: 0.7907
Test score: 0.44805118441581726
Test accuracy: 0.7906799912452698


## Exercise

In this exercise, we will illustrate:
- Preparing the data to use sequences of length 80 rather than length 30.  Does it improve the performance?
- Trying different values of the "max_features".  Does this  improve the performance?
- Trying smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?


In [13]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [14]:
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [16]:
rmsprop = keras.optimizers.legacy.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [17]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 6s 8ms/step - loss: 0.5932 - accuracy: 0.6792 - val_loss: 0.5114 - val_accuracy: 0.7462
Epoch 2/10
782/782 [==============================] - 6s 8ms/step - loss: 0.4371 - accuracy: 0.8001 - val_loss: 0.4391 - val_accuracy: 0.7949
Epoch 3/10
782/782 [==============================] - 6s 8ms/step - loss: 0.3720 - accuracy: 0.8373 - val_loss: 0.4119 - val_accuracy: 0.8130
Epoch 4/10
782/782 [==============================] - 6s 8ms/step - loss: 0.3326 - accuracy: 0.8588 - val_loss: 0.3832 - val_accuracy: 0.8261
Epoch 5/10
782/782 [==============================] - 6s 8ms/step - loss: 0.3060 - accuracy: 0.8720 - val_loss: 0.3775 - val_accuracy: 0.8314
Epoch 6/10
782/782 [==============================] - 6s 8ms/step - loss: 0.2873 - accuracy: 0.8804 - val_loss: 0.3652 - val_accuracy: 0.8374
Epoch 7/10
782/782 [==============================] - 7s 9ms/step - loss: 0.2731 - accuracy: 0.8873 - val_loss: 0.3608 - val_accuracy: 0.8401
Epoch 

In [18]:
max_features = 5000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [19]:
rnn_hidden_dim = 5
word_embedding_dim = 20
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [20]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [21]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 6s 7ms/step - loss: 0.6753 - accuracy: 0.5804 - val_loss: 0.6481 - val_accuracy: 0.6214
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 0.5649 - accuracy: 0.7108 - val_loss: 0.5414 - val_accuracy: 0.7204
Epoch 3/10
782/782 [==============================] - 5s 6ms/step - loss: 0.4919 - accuracy: 0.7631 - val_loss: 0.5135 - val_accuracy: 0.7375
Epoch 4/10
782/782 [==============================] - 5s 6ms/step - loss: 0.4433 - accuracy: 0.7940 - val_loss: 0.4623 - val_accuracy: 0.7818
Epoch 5/10
782/782 [==============================] - 5s 6ms/step - loss: 0.4056 - accuracy: 0.8196 - val_loss: 0.4418 - val_accuracy: 0.7923
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 0.3785 - accuracy: 0.8326 - val_loss: 0.4137 - val_accuracy: 0.8108
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 0.3607 - accuracy: 0.8431 - val_loss: 0.4074 - val_accuracy: 0.8153
Epoch 

In [22]:
# Out of curiosity, run for 10 more epochs
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 5s 6ms/step - loss: 0.3216 - accuracy: 0.8638 - val_loss: 0.3801 - val_accuracy: 0.8301
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 0.3148 - accuracy: 0.8661 - val_loss: 0.3856 - val_accuracy: 0.8302
Epoch 3/10
782/782 [==============================] - 5s 6ms/step - loss: 0.3091 - accuracy: 0.8711 - val_loss: 0.4010 - val_accuracy: 0.8164
Epoch 4/10
782/782 [==============================] - 5s 6ms/step - loss: 0.3039 - accuracy: 0.8726 - val_loss: 0.3742 - val_accuracy: 0.8338
Epoch 5/10
782/782 [==============================] - 5s 7ms/step - loss: 0.2997 - accuracy: 0.8750 - val_loss: 0.3780 - val_accuracy: 0.8328
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2951 - accuracy: 0.8760 - val_loss: 0.3815 - val_accuracy: 0.8327
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2920 - accuracy: 0.8773 - val_loss: 0.3742 - val_accuracy: 0.8350
Epoch 

---
### Machine Learning Foundation (C) 2020 IBM Corporation
